# Predicting with `np.dot` (`@`)

1. use case for dot product
2. one's column
3. matrix dot vector

$\begin{bmatrix}
1 & 2 \\ 3 & 4\\
\end{bmatrix}
\cdot
\begin{bmatrix}
10 \\ 1 \\
\end{bmatrix}$

In [1]:
import pandas as pd
import numpy as np

houses = pd.DataFrame([[2,1,1985],
                       [3,1,1998],
                       [4,3,2005],
                       [4,2,2020]],
                      columns=["beds", "baths", "year"])
houses

,beds,baths,year
0,2,1,1985
1,3,1,1998
2,4,3,2005
3,4,2,2020


In [2]:
# take row (as Series)
# return estimated price (in thousands)
def predict_price(house):
    return ((house["beds"]*42.3) + (house["baths"]*10) + 
            (house["year"]*1.67) - 3213)

predict_price(houses.iloc[0])

196.54999999999973

In [3]:
h = np.array([2,1,1985]).reshape(1,-1)
h

array([[   2,    1, 1985]])

In [4]:
c = np.array([42.3, 10, 1.67]).reshape(-1,1)
c

array([[42.3 ],
       [10.  ],
       [ 1.67]])

In [5]:
h @ c - 3213

array([[196.55]])

In [6]:
X = houses.values # underlying numpy array
X

array([[   2,    1, 1985],
       [   3,    1, 1998],
       [   4,    3, 2005],
       [   4,    2, 2020]])

In [7]:
print(X[0:1] @ c - 3213)
print(X[1:2] @ c - 3213)
print(X[2:3] @ c - 3213)
print(X[3:4] @ c - 3213)

[[196.55]]
[[260.56]]
[[334.55]]
[[349.6]]


In [8]:
X @ c - 3213

array([[196.55],
       [260.56],
       [334.55],
       [349.6 ]])

In [9]:
# can we eliminate the intercept b as a special case?

In [10]:
houses["constant_bias"] = 1
houses

,beds,baths,year,constant_bias
0,2,1,1985,1
1,3,1,1998,1
2,4,3,2005,1
3,4,2,2020,1


In [11]:
X = houses.values
X

array([[   2,    1, 1985,    1],
       [   3,    1, 1998,    1],
       [   4,    3, 2005,    1],
       [   4,    2, 2020,    1]])

In [12]:
c = np.array([42.3, 10, 1.67, -3213]).reshape(-1,1)
X @ c

array([[196.55],
       [260.56],
       [334.55],
       [349.6 ]])

In [13]:
X

array([[   2,    1, 1985,    1],
       [   3,    1, 1998,    1],
       [   4,    3, 2005,    1],
       [   4,    2, 2020,    1]])

# Fitting with `np.linalg.solve`

**Above:** we estimated house prices using a linear model based on the dot product as follows:

$Xc = y$

* $X$ (known) is a matrix with house features (from DataFrame)
* $c$ (known) is a vector of coefficients (our model parameters)
* $y$ (computed) are the prices

**Below:** what if X and y are know, and we want to find c?

In [14]:
houses = pd.DataFrame([[2,1,1985,196.55],
                       [3,1,1998,260.56],
                       [4,3,2005,334.55],
                       [4,2,2020,349.60]],
                      columns=["beds", "baths", "year", "price"])
houses

,beds,baths,year,price
0,2,1,1985,196.55
1,3,1,1998,260.56
2,4,3,2005,334.55
3,4,2,2020,349.60


If we assume price is linearly based on the features, with this equation:

* $beds*c_0 + baths*c_1 + year*c_2 + 1*c_3 = price$

Then we get four equations:

* $2*c_0 + 1*c_1 + 1985*c_2 + 1*c_3 = 196.55$
* $3*c_0 + 1*c_1 + 1998*c_2 + 1*c_3 = 260.56$
* $4*c_0 + 3*c_1 + 2005*c_2 + 1*c_3 = 334.55$
* $4*c_0 + 2*c_1 + 2020*c_2 + 1*c_3 = 349.60$

In [33]:
X = houses.loc[:, "beds":"year"].values
X = np.concatenate([X, np.ones(len(X)).reshape(-1, 1)], axis=1)
X

array([[2.000e+00, 1.000e+00, 1.985e+03, 1.000e+00],
       [3.000e+00, 1.000e+00, 1.998e+03, 1.000e+00],
       [4.000e+00, 3.000e+00, 2.005e+03, 1.000e+00],
       [4.000e+00, 2.000e+00, 2.020e+03, 1.000e+00]])

In [26]:
y = houses.loc[:, ["price"]].values
y

array([[196.55],
       [260.56],
       [334.55],
       [349.6 ]])

In [35]:
# y = X @ c
# c = solve(X, y)

c = np.linalg.solve(X, y)
c

array([[ 4.230e+01],
       [ 1.000e+01],
       [ 1.670e+00],
       [-3.213e+03]])

In [36]:
X @ c

array([[196.55],
       [260.56],
       [334.55],
       [349.6 ]])

In [37]:
np.array([[8,5,2021,1]]) @ c

array([[550.47]])

# Two Perspectives on `Matrix @ vector`

## Row Picture

Do dot product one row at a time.

$\begin{bmatrix}
4&5\\6&7\\8&9\\
\end{bmatrix}
\cdot
\begin{bmatrix}
2\\3\\
\end{bmatrix}
=
\begin{bmatrix}
(4*2)+(5*3)\\
(6*2)+(7*3)\\
(8*2)+(9*3)\\
\end{bmatrix}
=
\begin{bmatrix}
23\\
33\\
43\\
\end{bmatrix}
$

In [44]:
X = np.array([[4,5], [6,7], [8,9]])
c = np.array([2, 3]).reshape(-1,1)
X @ c

array([[23],
       [33],
       [43]])

In [47]:
def row_dot(M, v):
    v = v.reshape(-1)
    result = []
    for row in M:
        total = 0
        for i in range(len(row)):
            total += row[i] * v[i]
        result.append(total)
    return np.array(result).reshape(-1, 1)

row_dot(X, c)

array([[23],
       [33],
       [43]])

## Column Picture

$\begin{bmatrix}
c_0&c_1&c_2\\
\end{bmatrix}
\cdot
\begin{bmatrix}
x\\y\\z\\
\end{bmatrix}
=(c_0*x) + (c_1*y) + (c_2*z)
$

Dot product takes a **linear combination** of columns.

$\begin{bmatrix}
4&5\\6&7\\8&9\\
\end{bmatrix}
\cdot
\begin{bmatrix}
2\\3\\
\end{bmatrix}
=
\begin{bmatrix}
4\\6\\8\\
\end{bmatrix}*2
+
\begin{bmatrix}
5\\7\\9\\
\end{bmatrix}*3
=
\begin{bmatrix}
23\\
33\\
43\\
\end{bmatrix}
$

In [55]:
def col_dot(M, v):
    v = v.reshape(-1)
    result = np.zeros(len(M)).reshape(-1, 1)
    for colidx in range(M.shape[1]):
        column = M[:, colidx:colidx+1]
        weight = v[colidx]
        result += column * weight
    return result

col_dot(X, c)

array([[23.],
       [33.],
       [43.]])